In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [2]:
from datasets import load_dataset

# Collect the training tasks

* Paraphrasing: https://huggingface.co/datasets/GEM/opusparcus
* Translation: https://huggingface.co/datasets/open_subtitles + news_commentary? + tatoeba?
* Detox: ordinary data

In [3]:
opus_para_en = load_dataset("GEM/opusparcus", "en.80")
opus_para_en

Reusing dataset opusparcus (/home/dale/.cache/huggingface/datasets/GEM___opusparcus/en.80/1.0.0/79d36ae4eced4f3c2c5a2ab9f94a584de7adca957186408d33798d0d87b018f2)


  0%|          | 0/5 [00:00<?, ?it/s]

DatasetDict({
    test: Dataset({
        features: ['lang', 'input', 'target', 'annot_score', 'gem_id', 'references'],
        num_rows: 982
    })
    validation: Dataset({
        features: ['lang', 'input', 'target', 'annot_score', 'gem_id', 'references'],
        num_rows: 1015
    })
    test.full: Dataset({
        features: ['lang', 'input', 'target', 'annot_score', 'gem_id', 'references'],
        num_rows: 1445
    })
    validation.full: Dataset({
        features: ['lang', 'input', 'target', 'annot_score', 'gem_id', 'references'],
        num_rows: 1455
    })
    train: Dataset({
        features: ['lang', 'input', 'target', 'annot_score', 'gem_id', 'references'],
        num_rows: 5200000
    })
})

In [4]:
opus_para_ru = load_dataset("GEM/opusparcus", "ru.80")
opus_para_ru

Reusing dataset opusparcus (/home/dale/.cache/huggingface/datasets/GEM___opusparcus/ru.80/1.0.0/79d36ae4eced4f3c2c5a2ab9f94a584de7adca957186408d33798d0d87b018f2)


  0%|          | 0/5 [00:00<?, ?it/s]

DatasetDict({
    test: Dataset({
        features: ['lang', 'input', 'target', 'annot_score', 'gem_id', 'references'],
        num_rows: 1068
    })
    validation: Dataset({
        features: ['lang', 'input', 'target', 'annot_score', 'gem_id', 'references'],
        num_rows: 1020
    })
    test.full: Dataset({
        features: ['lang', 'input', 'target', 'annot_score', 'gem_id', 'references'],
        num_rows: 1855
    })
    validation.full: Dataset({
        features: ['lang', 'input', 'target', 'annot_score', 'gem_id', 'references'],
        num_rows: 1854
    })
    train: Dataset({
        features: ['lang', 'input', 'target', 'annot_score', 'gem_id', 'references'],
        num_rows: 2300000
    })
})

In [6]:
import random

In [7]:
random.choice(opus_para_ru['train'])

{'lang': 'ru',
 'input': 'Ты прости.',
 'target': 'Сожалею.',
 'annot_score': 0.0,
 'gem_id': 'gem-opusparcus-train-80334819',
 'references': ['Сожалею.']}

In [8]:
opensub = load_dataset("open_subtitles", lang1="en", lang2='ru')
opensub

Using custom data configuration en-ru-lang1=en,lang2=ru
Reusing dataset open_subtitles (/home/dale/.cache/huggingface/datasets/open_subtitles/en-ru-lang1=en,lang2=ru/0.0.0/c1ec973ca4b6e588740d8f167cc0e24ea3f626e70bc7ffe467e944730500e198)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'meta', 'translation'],
        num_rows: 25910105
    })
})

In [9]:
random.choice(opensub['train'])

{'id': '22222006',
 'meta': {'year': 2015,
  'imdbId': 4516410,
  'subtitleId': {'en': 6571424, 'ru': 6126244},
  'sentenceIds': {'en': [543], 'ru': [522]}},
 'translation': {'en': '- Look here.', 'ru': '- Посмотри'}}

In [10]:
news_commentary = load_dataset("news_commentary", lang1="en", lang2='ru')
news_commentary

Using custom data configuration en-ru-lang1=en,lang2=ru
Reusing dataset news_commentary (/home/dale/.cache/huggingface/datasets/news_commentary/en-ru-lang1=en,lang2=ru/0.0.0/cfab724ce975dc2da51cdae45302389860badc88b74db8570d561ced6004f8b4)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 190104
    })
})

In [11]:
random.choice(news_commentary['train'])

{'id': '96821',
 'translation': {'en': 'For the first time in its troubled history, Romania has nothing to fear from its neighbours.',
  'ru': 'Впервые за всю свою беспокойную историю Румынии нечего опасаться своих соседей.'}}

In [12]:
tatoeba = load_dataset("tatoeba", lang1="en", lang2='ru')
tatoeba

Using custom data configuration en-ru-lang1=en,lang2=ru
Reusing dataset tatoeba (/home/dale/.cache/huggingface/datasets/tatoeba/en-ru-lang1=en,lang2=ru/0.0.0/b3ea9c6bb2af47699c5fc0a155643f5a0da287c7095ea14824ee0a8afd74daf6)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 523656
    })
})

In [13]:
random.choice(tatoeba['train'])

{'id': '166210',
 'translation': {'en': "It's beautiful here.", 'ru': 'Здесь красиво.'}}

In [14]:
def get_paraphrase_task(batch_size=1):
    task = 'paraphrase: '
    if random.random() < 0.5:
        src = opus_para_en
        src_id, tgt_id = 'en_XX', 'en_XX'
    else:
        src = opus_para_ru
        src_id, tgt_id = 'ru_RU', 'ru_RU'
    x = []
    y = []
    for _ in range(batch_size):
        item = random.choice(src['train'])
        xx, yy = item['input'], item['target']
        if random.random() < 0.5:
            xx, yy = yy, xx
        x.append(task + xx)
        y.append(yy)
    return x, y, src_id, tgt_id
        
get_paraphrase_task(1)

(['paraphrase: Я главный агент Уорен Расс.'],
 ['Здесь я главный.'],
 'ru_RU',
 'ru_RU')

In [15]:
def get_translate_task(batch_size=1):
    task = 'translate: '
    src = random.choice([tatoeba, opensub, news_commentary])
    src_id, tgt_id = 'ru_RU', 'en_XX'
    if random.random() < 0.5:
        src_id, tgt_id = tgt_id, src_id
    x = []
    y = []
    for _ in range(batch_size):
        item = random.choice(src['train'])
        x.append(task + item['translation'][src_id[:2]])
        y.append(item['translation'][tgt_id[:2]])
    return x, y, src_id, tgt_id

get_translate_task(2)

(['translate: Давайте начнем отсюда.',
  'translate: Как давно вы здесь живёте?'],
 ["Let's start here.", 'How long have you been living here?'],
 'ru_RU',
 'en_XX')

In [16]:
import pandas as pd

In [17]:
detox_en_train = pd.read_csv('../data/english_data/en_data.csv', sep='\t')

In [88]:
from sklearn.model_selection import train_test_split
detox_train, detox_val = train_test_split(detox_en_train, random_state=1, test_size=500)

In [89]:
%%time
detox_train.sample(3)

CPU times: user 1.6 ms, sys: 288 µs, total: 1.89 ms
Wall time: 1.77 ms


,idx,toxic_comment,neutral_comment,toxicity_score,dataset,toxic,confidence_toxic,is_match,confidence_is_match
6395,83656,don 't bother yourself with that shit .,don 't bother yourself with that situation,0.998556,twitter,False,0.9985,True,0.9862
9786,54986,u just gon recite the whole damn song huh ? ?,u just gon recite the whole song,0.996870,twitter,False,0.9985,True,0.9900
6291,7659,no respect from the bitches ?,no respect from the girls ?,0.996808,twitter,False,0.9996,True,0.9302


In [90]:
def get_detox_task(batch_size=1):
    task = 'detox: '
    sample = detox_train.sample(batch_size)
    src_id, tgt_id = 'en_XX', 'en_XX'
    x = []
    y = []
    for row in sample.itertuples():
        x.append(task + row.toxic_comment)
        y.append(row.neutral_comment)
    return x, y, src_id, tgt_id

get_detox_task(2)

(['detox: you rap like a fucking 40 year old bitch .',
  'detox: the shit you just said !'],
 ['Your rap is old.', 'What did you just say?'],
 'en_XX',
 'en_XX')

In [24]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [25]:
model_name = '/home/dale/models/detox-parallel/mbart_5000_EN'
tokenizer = AutoTokenizer.from_pretrained('facebook/mbart-large-50')

In [26]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).cuda();

In [27]:
def get_batch(batch_size=3):
    x = []
    y = []
    bs = batch_size // 3
    for generator in [get_detox_task, get_paraphrase_task, get_translate_task]:
        for _ in range(bs):
            xx, yy, src_id, tgt_id = generator(1)
            tokenizer.src_lang, tokenizer.tgt_lang = src_id, tgt_id
            x.append(tokenizer(xx))
            with tokenizer.as_target_tokenizer():
                y.append(tokenizer(yy))
    return x, y

In [85]:
x, y = get_batch(3)
x, y

([{'input_ids': [[250004, 162770, 12, 1836, 1221, 765, 70, 3428, 68818, 21771, 2685, 47, 63284, 100592, 13, 831, 242, 18, 54, 93669, 1672, 442, 6, 5, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]},
  {'input_ids': [[250021, 121, 179665, 184, 12, 1509, 81880, 21136, 5, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]},
  {'input_ids': [[250004, 3900, 19309, 12, 138521, 5844, 538, 4, 642, 621, 35782, 214, 23, 70, 78431, 26548, 8999, 1926, 1505, 5, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}],
 [{'input_ids': [[250004, 10660, 1221, 765, 70, 3428, 68818, 21771, 2685, 47, 63284, 82739, 831, 242, 18, 54, 23937, 1672, 442, 5, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]},
  {'input_ids': [[250021, 1509, 81880, 21136, 4, 112613, 5, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1]]},
  {'input_ids': [[250021, 417, 130390, 419, 71459, 103, 2176, 

In [86]:
for xx, yy in zip(x, y):
    print(tokenizer.decode(xx['input_ids'][0]))
    print(tokenizer.decode(yy['input_ids'][0]))

en_XX detox: they will have the troops already there to ensure ukraine can 't do shit about it.</s>
en_XX They will have the troops already there to ensure Ukraine can 't do anything about it.</s>
ru_RU paraphrase: Я люблю тебя.</s>
ru_RU Я люблю тебя, Мама.</s>
en_XX translate: Collectively, we are failing in the fight against world hunger.</s>
ru_RU В глобальном масштабе мы проигрываем войну с мировым голодом.</s>


In [75]:
tokenizer.pad([{k: v[0] for k, v in item.items()} for item in x])

{'input_ids': [[250004, 162770, 12, 764, 1556, 11343, 194663, 1884, 142, 68649, 3853, 20271, 70, 4568, 99656, 28021, 6, 5, 2], [250004, 121, 179665, 184, 12, 10868, 32, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [250021, 3900, 19309, 12, 20, 16434, 77, 67008, 4, 414, 784, 111379, 38, 2, 1, 1, 1, 1, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]]}

In [34]:
import transformers
transformers.logging.set_verbosity_error()

In [31]:
from transformers.optimization import Adafactor
optimizer = Adafactor(model.parameters(), scale_parameter=False, relative_step=False, lr=1e-5, clip_threshold=1.0)

In [32]:
from torch.optim.lr_scheduler import LambdaLR

num_warmup_steps = 1000

def lr_lambda(current_step: int):
    if current_step < num_warmup_steps:
        return float(current_step) / float(max(1.0, num_warmup_steps))
    return 1.0

scheduler = LambdaLR(optimizer, lr_lambda)

In [33]:
model.train()
model.gradient_checkpointing_enable()

In [37]:
from tqdm.auto import tqdm, trange

In [59]:
import torch
import gc

def cleanup():
    gc.collect()
    torch.cuda.empty_cache()

cleanup()

In [73]:
gradient_steps = 4
batch_size = 6
window = 1000
report_steps = 1000
cleanup_steps = 100
save_path = '/home/dale/models/detox-parallel/bart-multitask-v1'

In [57]:
ewm_loss = 4

In [64]:
cleanup()

With ~6 iterations per second, 1M iteration takes 1000000/6/60/60=45 hours.

In [ ]:
model.train()
tq = trange(598943, 1_000_000)
cleanup()

for i in tq:
    x, y = get_batch(batch_size)
    xpad = tokenizer.pad([{k: v[0] for k, v in item.items()} for item in x])
    ypad = tokenizer.pad([{k: v[0] for k, v in item.items()} for item in y])
    
    try:
        labels = torch.tensor(ypad['input_ids'], device=model.device)
        labels[labels==tokenizer.pad_token_id] = -100
        loss = model(
            input_ids=torch.tensor(xpad['input_ids'], device=model.device),
            attention_mask=torch.tensor(xpad['attention_mask'], device=model.device),
            labels=labels,
        ).loss
        loss.backward()
        if i % gradient_steps == 0:
            optimizer.step()
            optimizer.zero_grad(set_to_none=True)
            scheduler.step()
    except RuntimeError as e:
        print('error', i, e)
        loss = None
        optimizer.zero_grad(set_to_none=True)
        cleanup()
        continue

    w = 1 / max(1, min(i, window))
    ewm_loss = ewm_loss * (1-w) + loss.item() * w
    tq.set_description(f'{ewm_loss:3.4f}')

    if i > 0 and i % report_steps == 0:
        print('step', i, 'loss', ewm_loss, )
        if i > 0:
            model.save_pretrained(save_path)
            tokenizer.save_pretrained(save_path)
    if i % cleanup_steps == 0:
        cleanup()

    

  0%|          | 0/401057 [00:00<?, ?it/s]

step 599000 loss 0.791350449541119
step 600000 loss 0.7908788145488161
error 600904 CUDA out of memory. Tried to allocate 906.00 MiB (GPU 0; 10.76 GiB total capacity; 8.02 GiB already allocated; 785.69 MiB free; 8.87 GiB reserved in total by PyTorch)
step 601000 loss 0.7923423596750206
step 602000 loss 0.7940523141718425
error 602326 CUDA out of memory. Tried to allocate 928.00 MiB (GPU 0; 10.76 GiB total capacity; 8.14 GiB already allocated; 799.69 MiB free; 8.85 GiB reserved in total by PyTorch)
step 603000 loss 0.790953011557408
error 603566 CUDA out of memory. Tried to allocate 870.00 MiB (GPU 0; 10.76 GiB total capacity; 8.08 GiB already allocated; 815.69 MiB free; 8.84 GiB reserved in total by PyTorch)
step 604000 loss 0.8040739516727109
step 605000 loss 0.7983863036420874
step 606000 loss 0.8064560542508245
step 607000 loss 0.8109014954943938
step 608000 loss 0.8060132506587476
step 609000 loss 0.8090561751829025


In [154]:
print(i, ewm_loss)
# step 141000 loss 0.9336458999525007
# 598943 0.7913156676625156
# 999999 0.7528545096685079

999999 0.7528545096685079


```
step 1000 loss 1.5705595690906056
error 1146 CUDA out of memory. Tried to allocate 1.35 GiB (GPU 0; 10.76 GiB total capacity; 8.85 GiB already allocated; 475.69 MiB free; 9.17 GiB reserved in total by PyTorch)
step 2000 loss 1.5289358124251258
step 3000 loss 1.4603064465819475
step 4000 loss 1.4280587719245064
error 4731 CUDA out of memory. Tried to allocate 916.00 MiB (GPU 0; 10.76 GiB total capacity; 8.11 GiB already allocated; 801.69 MiB free; 8.85 GiB reserved in total by PyTorch)
error 4840 CUDA out of memory. Tried to allocate 978.00 MiB (GPU 0; 10.76 GiB total capacity; 7.16 GiB already allocated; 867.69 MiB free; 8.79 GiB reserved in total by PyTorch)
step 5000 loss 1.3723411050480274
step 6000 loss 1.3470262561429978
step 7000 loss 1.3337626059557242
step 8000 loss 1.3178974970698947
error 8569 CUDA out of memory. Tried to allocate 3.58 GiB (GPU 0; 10.76 GiB total capacity; 6.39 GiB already allocated; 3.11 GiB free; 6.53 GiB reserved in total by PyTorch)
step 9000 loss 1.3109540848422108
step 10000 loss 1.3099768834329912
step 11000 loss 1.2894572959109634
error 11954 CUDA out of memory. Tried to allocate 1.26 GiB (GPU 0; 10.76 GiB total capacity; 8.24 GiB already allocated; 633.69 MiB free; 9.02 GiB reserved in total by PyTorch)
step 12000 loss 1.2661666174786903
step 13000 loss 1.2544034908444262
step 14000 loss 1.2537943159628
step 15000 loss 1.2417901334227581
error 15970 CUDA out of memory. Tried to allocate 780.00 MiB (GPU 0; 10.76 GiB total capacity; 7.67 GiB already allocated; 767.69 MiB free; 8.88 GiB reserved in total by PyTorch)
step 16000 loss 1.2500597551439607
step 17000 loss 1.2311876436254687
step 18000 loss 1.2300539663708037
step 19000 loss 1.214656587177018
step 20000 loss 1.209378143381565
step 21000 loss 1.2033179850564317
step 22000 loss 1.1968057551352824
step 23000 loss 1.176260400882675
error 23584 CUDA out of memory. Tried to allocate 978.00 MiB (GPU 0; 10.76 GiB total capacity; 7.28 GiB already allocated; 829.69 MiB free; 8.82 GiB reserved in total by PyTorch)
step 24000 loss 1.1754935866907865
error 24896 CUDA out of memory. Tried to allocate 144.00 MiB (GPU 0; 10.76 GiB total capacity; 9.44 GiB already allocated; 59.69 MiB free; 9.58 GiB reserved in total by PyTorch)
step 25000 loss 1.1754272553817264
step 26000 loss 1.1656607958886298
step 27000 loss 1.1570394344844919
step 28000 loss 1.1558873855161609
step 29000 loss 1.165974972071494
step 30000 loss 1.1599093132613605
step 31000 loss 1.1541310660451913
error 31282 CUDA out of memory. Tried to allocate 1.19 GiB (GPU 0; 10.76 GiB total capacity; 8.76 GiB already allocated; 287.69 MiB free; 9.35 GiB reserved in total by PyTorch)
step 32000 loss 1.1364641991741162
step 33000 loss 1.132973628288517
error 33624 CUDA out of memory. Tried to allocate 144.00 MiB (GPU 0; 10.76 GiB total capacity; 9.44 GiB already allocated; 79.69 MiB free; 9.56 GiB reserved in total by PyTorch)
step 34000 loss 1.1286950876108275
step 35000 loss 1.1291851942061415
step 36000 loss 1.1235886658742347
step 37000 loss 1.1246682009640214
step 38000 loss 1.1135467563562322
step 39000 loss 1.1149872643497183
step 40000 loss 1.1017778124035793
step 41000 loss 1.1060003889083563
error 41735 CUDA out of memory. Tried to allocate 870.00 MiB (GPU 0; 10.76 GiB total capacity; 8.07 GiB already allocated; 859.69 MiB free; 8.79 GiB reserved in total by PyTorch)
step 42000 loss 1.1078476631524794
step 43000 loss 1.100026474680707
step 44000 loss 1.095670024500316
step 45000 loss 1.109476816996912
step 46000 loss 1.1119795778634898
step 47000 loss 1.1311553555564193
step 48000 loss 1.113347318553216
step 49000 loss 1.1016645401565517
step 50000 loss 1.1220686658058219
error 50454 CUDA out of memory. Tried to allocate 916.00 MiB (GPU 0; 10.76 GiB total capacity; 7.88 GiB already allocated; 821.69 MiB free; 8.83 GiB reserved in total by PyTorch)
step 51000 loss 1.1214904008655675
step 52000 loss 1.0854113000333645
step 53000 loss 1.088154120166232
step 54000 loss 1.093514120254794
step 55000 loss 1.0922690118309737
error 55998 CUDA out of memory. Tried to allocate 952.00 MiB (GPU 0; 10.76 GiB total capacity; 8.01 GiB already allocated; 695.69 MiB free; 8.96 GiB reserved in total by PyTorch)
step 56000 loss 1.0800373847786937
step 57000 loss 1.0920900005165737
step 58000 loss 1.0736909533657972
step 59000 loss 1.0752592100786067
step 60000 loss 1.0637662286778657
step 61000 loss 1.0654742857820474
error 61504 CUDA out of memory. Tried to allocate 946.00 MiB (GPU 0; 10.76 GiB total capacity; 7.89 GiB already allocated; 757.69 MiB free; 8.89 GiB reserved in total by PyTorch)
step 62000 loss 1.0565901908696624
step 63000 loss 1.057395690647219
step 64000 loss 1.0477948828239614
step 65000 loss 1.0296297263268868
error 65595 CUDA out of memory. Tried to allocate 1.07 GiB (GPU 0; 10.76 GiB total capacity; 8.30 GiB already allocated; 781.69 MiB free; 8.87 GiB reserved in total by PyTorch)
step 66000 loss 1.0449434271303466
step 67000 loss 1.0265650949758665
step 68000 loss 1.025196188228675
error 68646 CUDA out of memory. Tried to allocate 1.03 GiB (GPU 0; 10.76 GiB total capacity; 8.18 GiB already allocated; 1011.69 MiB free; 8.65 GiB reserved in total by PyTorch)
step 69000 loss 1.040014132410023
step 70000 loss 1.0401423453181597
step 71000 loss 1.0255363661922454
step 72000 loss 1.025135515281328
step 73000 loss 1.0365278080704476
step 74000 loss 1.0183107856778406
error 74386 CUDA out of memory. Tried to allocate 894.00 MiB (GPU 0; 10.76 GiB total capacity; 8.19 GiB already allocated; 739.69 MiB free; 8.91 GiB reserved in total by PyTorch)
step 75000 loss 1.0290407528180614
step 76000 loss 1.028850245665216
step 77000 loss 1.0279383226408405
step 78000 loss 1.022980517921964
step 79000 loss 1.0177867619738628
step 80000 loss 1.0196264535815993
step 81000 loss 1.0091859735396909
step 82000 loss 1.0045839575710827
step 83000 loss 0.9986900074678753
step 84000 loss 0.9907349173959362
error 84856 CUDA out of memory. Tried to allocate 934.00 MiB (GPU 0; 10.76 GiB total capacity; 7.84 GiB already allocated; 743.69 MiB free; 8.91 GiB reserved in total by PyTorch)
step 85000 loss 1.002218034634077
step 86000 loss 0.9975985926845609
error 86519 CUDA out of memory. Tried to allocate 1.04 GiB (GPU 0; 10.76 GiB total capacity; 8.31 GiB already allocated; 475.69 MiB free; 9.17 GiB reserved in total by PyTorch)
error 86791 CUDA out of memory. Tried to allocate 946.00 MiB (GPU 0; 10.76 GiB total capacity; 8.18 GiB already allocated; 531.69 MiB free; 9.12 GiB reserved in total by PyTorch)
step 87000 loss 0.999982904219524
error 87064 CUDA out of memory. Tried to allocate 968.00 MiB (GPU 0; 10.76 GiB total capacity; 8.10 GiB already allocated; 763.69 MiB free; 8.89 GiB reserved in total by PyTorch)
error 87252 CUDA out of memory. Tried to allocate 1.14 GiB (GPU 0; 10.76 GiB total capacity; 8.50 GiB already allocated; 175.69 MiB free; 9.46 GiB reserved in total by PyTorch)
step 88000 loss 1.0042518996963314
error 88427 CUDA out of memory. Tried to allocate 912.00 MiB (GPU 0; 10.76 GiB total capacity; 7.85 GiB already allocated; 861.69 MiB free; 8.79 GiB reserved in total by PyTorch)
step 89000 loss 0.9853234168833125
step 90000 loss 0.9924546283292504
step 91000 loss 0.9888159920285438
step 92000 loss 0.9908827874992283
step 93000 loss 0.9790343402052905
error 93459 CUDA out of memory. Tried to allocate 922.00 MiB (GPU 0; 10.76 GiB total capacity; 7.89 GiB already allocated; 769.69 MiB free; 8.88 GiB reserved in total by PyTorch)
step 94000 loss 0.9948122169551259
error 94568 CUDA out of memory. Tried to allocate 1.28 GiB (GPU 0; 10.76 GiB total capacity; 8.75 GiB already allocated; 93.69 MiB free; 9.54 GiB reserved in total by PyTorch)
error 94798 CUDA out of memory. Tried to allocate 734.00 MiB (GPU 0; 10.76 GiB total capacity; 7.62 GiB already allocated; 599.69 MiB free; 9.05 GiB reserved in total by PyTorch)
step 95000 loss 0.9982311906577338
step 96000 loss 0.9977218772548391
step 97000 loss 0.9998095549900895
step 98000 loss 0.9892420303142296
step 99000 loss 1.0010712033357734
error 99226 CUDA out of memory. Tried to allocate 968.00 MiB (GPU 0; 10.76 GiB total capacity; 8.47 GiB already allocated; 797.69 MiB free; 8.86 GiB reserved in total by PyTorch)
step 100000 loss 0.993382955842513
step 101000 loss 0.989972051978362
step 102000 loss 0.9816771483956006
error 102955 CUDA out of memory. Tried to allocate 814.00 MiB (GPU 0; 10.76 GiB total capacity; 7.56 GiB already allocated; 777.69 MiB free; 8.88 GiB reserved in total by PyTorch)
step 103000 loss 0.9892074373843217
error 103353 CUDA out of memory. Tried to allocate 448.00 MiB (GPU 0; 10.76 GiB total capacity; 8.67 GiB already allocated; 367.69 MiB free; 9.28 GiB reserved in total by PyTorch)
error 103360 CUDA out of memory. Tried to allocate 928.00 MiB (GPU 0; 10.76 GiB total capacity; 7.79 GiB already allocated; 827.69 MiB free; 8.83 GiB reserved in total by PyTorch)
error 103459 CUDA out of memory. Tried to allocate 1.45 GiB (GPU 0; 10.76 GiB total capacity; 9.45 GiB already allocated; 81.69 MiB free; 9.55 GiB reserved in total by PyTorch)
step 104000 loss 0.9956771865461124
step 105000 loss 0.9826513917788725
step 106000 loss 0.9809061708308737
step 107000 loss 0.9690858024975915
error 107880 CUDA out of memory. Tried to allocate 928.00 MiB (GPU 0; 10.76 GiB total capacity; 7.89 GiB already allocated; 749.69 MiB free; 8.90 GiB reserved in total by PyTorch)
step 108000 loss 0.9768115332263623
step 109000 loss 0.9730555710098093
error 109867 CUDA out of memory. Tried to allocate 978.00 MiB (GPU 0; 10.76 GiB total capacity; 7.98 GiB already allocated; 55.69 MiB free; 9.58 GiB reserved in total by PyTorch)
step 110000 loss 0.9588704389592169
step 111000 loss 0.96905091931659
step 112000 loss 0.9584886363872099
step 113000 loss 0.9626811534210103
error 113832 CUDA out of memory. Tried to allocate 986.00 MiB (GPU 0; 10.76 GiB total capacity; 8.36 GiB already allocated; 681.69 MiB free; 8.97 GiB reserved in total by PyTorch)
step 114000 loss 0.9652874172750078
error 114256 CUDA out of memory. Tried to allocate 740.00 MiB (GPU 0; 10.76 GiB total capacity; 7.62 GiB already allocated; 725.69 MiB free; 8.93 GiB reserved in total by PyTorch)
step 115000 loss 0.9702503596144891
step 116000 loss 0.9834469998620671
step 117000 loss 0.9565716304994859
error 117802 CUDA out of memory. Tried to allocate 1.05 GiB (GPU 0; 10.76 GiB total capacity; 8.48 GiB already allocated; 115.69 MiB free; 9.52 GiB reserved in total by PyTorch)
step 118000 loss 0.9472308477471018
step 119000 loss 0.9432499939710902
step 120000 loss 0.9487864710832014
error 120352 CUDA out of memory. Tried to allocate 928.00 MiB (GPU 0; 10.76 GiB total capacity; 7.89 GiB already allocated; 765.69 MiB free; 8.89 GiB reserved in total by PyTorch)
step 121000 loss 0.9565588829385917
error 121728 CUDA out of memory. Tried to allocate 894.00 MiB (GPU 0; 10.76 GiB total capacity; 8.00 GiB already allocated; 775.69 MiB free; 8.88 GiB reserved in total by PyTorch)
step 122000 loss 0.9532983707522799
error 122227 CUDA out of memory. Tried to allocate 980.00 MiB (GPU 0; 10.76 GiB total capacity; 8.09 GiB already allocated; 605.69 MiB free; 9.04 GiB reserved in total by PyTorch)
step 123000 loss 0.9446579631148261
error 123827 CUDA out of memory. Tried to allocate 1.28 GiB (GPU 0; 10.76 GiB total capacity; 9.21 GiB already allocated; 227.69 MiB free; 9.41 GiB reserved in total by PyTorch)
step 124000 loss 0.9559603861394274
step 125000 loss 0.9535240063586068
error 125740 CUDA out of memory. Tried to allocate 916.00 MiB (GPU 0; 10.76 GiB total capacity; 7.93 GiB already allocated; 835.69 MiB free; 8.82 GiB reserved in total by PyTorch)
error 125771 CUDA out of memory. Tried to allocate 978.00 MiB (GPU 0; 10.76 GiB total capacity; 7.13 GiB already allocated; 937.69 MiB free; 8.72 GiB reserved in total by PyTorch)
step 126000 loss 0.9491991817012265
error 126478 CUDA out of memory. Tried to allocate 980.00 MiB (GPU 0; 10.76 GiB total capacity; 8.05 GiB already allocated; 749.69 MiB free; 8.90 GiB reserved in total by PyTorch)
step 127000 loss 0.9462021857845146
step 128000 loss 0.9454268055951242
step 129000 loss 0.9485337868782098
step 130000 loss 0.9403961820295327
step 131000 loss 0.9420713849797546
step 132000 loss 0.9559321375595082
error 132104 CUDA out of memory. Tried to allocate 978.00 MiB (GPU 0; 10.76 GiB total capacity; 6.98 GiB already allocated; 919.69 MiB free; 8.74 GiB reserved in total by PyTorch)
step 133000 loss 0.9514717640636141
step 134000 loss 0.9486987142211305
```

```
step 599000 loss 0.791350449541119
step 600000 loss 0.7908788145488161
error 600904 CUDA out of memory. Tried to allocate 906.00 MiB (GPU 0; 10.76 GiB total capacity; 8.02 GiB already allocated; 785.69 MiB free; 8.87 GiB reserved in total by PyTorch)
step 601000 loss 0.7923423596750206
step 602000 loss 0.7940523141718425
error 602326 CUDA out of memory. Tried to allocate 928.00 MiB (GPU 0; 10.76 GiB total capacity; 8.14 GiB already allocated; 799.69 MiB free; 8.85 GiB reserved in total by PyTorch)
step 603000 loss 0.790953011557408
error 603566 CUDA out of memory. Tried to allocate 870.00 MiB (GPU 0; 10.76 GiB total capacity; 8.08 GiB already allocated; 815.69 MiB free; 8.84 GiB reserved in total by PyTorch)
step 604000 loss 0.8040739516727109
step 605000 loss 0.7983863036420874
step 606000 loss 0.8064560542508245
step 607000 loss 0.8109014954943938
step 608000 loss 0.8060132506587476
step 609000 loss 0.8090561751829025
```

In [155]:
model.eval();

In [156]:
tokenizer.src_lang = 'ru_RU'

In [157]:
tokenizer.src_lang = 'en_XX'

In [158]:
tokenizer('привет')

{'input_ids': [250004, 146038, 2], 'attention_mask': [1, 1, 1]}

In [159]:
tokenizer.convert_tokens_to_ids('en_XX')

250004

In [160]:
def paraphrase(
    text, model, tokenizer, 
    n=None, 
    max_length="auto", 
    beams=5,
    src_lang='en_XX',
    tgt_lang='en_XX',
    **kwargs
):
    tokenizer.src_lang = src_lang
    tokenizer.tgt_lang = tgt_lang
    texts = [text] if isinstance(text, str) else text
    tokenizer.tgt_lang
    inputs = tokenizer(texts, return_tensors="pt", padding=True)["input_ids"].to(
        model.device
    )

    if max_length == "auto":
        max_length = inputs.shape[1] + 10

    result = model.generate(
        inputs,
        num_return_sequences=n or 1,
        do_sample=False,
        temperature=1.0,
        repetition_penalty=10.0,
        max_length=max_length,
        min_length=int(0.5 * max_length),
        num_beams=beams,
        forced_bos_token_id=tokenizer.convert_tokens_to_ids(tgt_lang),
    )
    texts = [tokenizer.decode(r, skip_special_tokens=True) for r in result]

    if not n and isinstance(text, str):
        return texts[0]
    return texts[0]

In [161]:
print(paraphrase('paraphrase: I like to play with my nice dog.', model, tokenizer))

I like to play with my dog, too.


In [162]:
print(paraphrase('translate: I like to play with my nice dog.', model, tokenizer, tgt_lang='ru_RU'))

Мне нравится играть со своей миленькой собакой.


In [163]:
print(paraphrase('translate: I hate to play with my fucking dog.', model, tokenizer, tgt_lang='ru_RU'))

Ненавижу играть со своей чертовой собакой.


In [164]:
print(paraphrase('translate: Ненавижу играть со своей долбаной собакой.', model, tokenizer, src_lang='ru_RU'))

I hate playing with my fucking dog. It's not a good thing.


In [165]:
print(paraphrase('paraphrase: I hate to play with my fucking dog.', model, tokenizer, tgt_lang='en_XX'))

I don't like to play with my dog.


In [166]:
print(paraphrase('detox: I hate to play with my fucking dog.', model, tokenizer, tgt_lang='en_XX'))

I don't like to play with my dog.


In [167]:
print(paraphrase('detox: Ненавижу играть со своей долбаной собакой.', model, tokenizer, src_lang='ru_RU', tgt_lang='ru_RU'))

Терпеть не могу играть со своей собакой.


In [168]:
test_data = pd.read_csv('../data/russian_data/test.tsv', sep='\t')
test_inputs = test_data["toxic_comment"].values.tolist()

In [169]:
test_outputs = [paraphrase('detox: ' + text, model, tokenizer, src_lang='ru_RU', tgt_lang='ru_RU') for text in tqdm(test_inputs)]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [170]:
path = '../results/multitask-1M/'  # partial training results; need to train further.
if not os.path.exists(path):
    os.mkdir(path)

In [171]:
with open(path + 'results_ru.txt', 'w') as f:
    for line in test_outputs:
        f.write(line + '\n')

Preliminary results
```

python evaluate_ru.py \
    --result_filename scores \
    --input_dir results/multitask-135K \
    --output_dir results
    
Style accuracy:       0.22049465775489807
Meaning preservation: 0.9518442153930664
Joint fluency:        -0.0548839345574379
Joint score:          -0.01705724559724331
Scores after calibration:
Style accuracy:       0.2984451949596405
Meaning preservation: 0.9277663826942444
Joint fluency:        0.9368835091590881
Joint score:          0.24435113370418549
```

Full results (1m steps)
```
python evaluate_ru.py \
    --result_filename scores \
    --input_dir results/multitask-1M \
    --output_dir results
    
Style accuracy:       0.25191155076026917
Meaning preservation: 0.948351263999939
Joint fluency:        -0.06296750158071518
Joint score:          -0.021249013021588326
Scores after calibration:
Style accuracy:       0.32672038674354553
Meaning preservation: 0.9225268363952637
Joint fluency:        0.9275873899459839
Joint score:          0.261173278093338
```

In [175]:
#with open('../results/copypaste/results_ru.txt', 'w') as f:
#    for line in test_inputs:
#        f.write(line+'\n')

# Add a Russian (translated) detox dataset as well

In [180]:
detox_en2ru = pd.read_csv('detox_en2ru_yandex.tsv', sep='\t')

In [181]:
detox_en2ru_filtered = detox_en2ru[
    (detox_en2ru.edit_distance_ru >= detox_en2ru.edit_distance_en.quantile(0.01)) 
    & (detox_en2ru.edit_distance_ru <= detox_en2ru.edit_distance_en.quantile(0.99)) 
    & (detox_en2ru.edit_sim_ru >= detox_en2ru.edit_sim_en.quantile(0.01)) 
    & (detox_en2ru.edit_sim_ru <= detox_en2ru.edit_sim_en.quantile(0.99))
]

print(detox_en2ru.shape)
print(detox_en2ru_filtered.shape)

(19766, 15)
(18488, 15)


In [183]:
detox_en2ru_filtered.sample(3)

,idx,toxic_comment,neutral_comment,toxicity_score,dataset,toxic,confidence_toxic,is_match,confidence_is_match,toxic_ru,neutral_ru,edit_distance_en,edit_distance_ru,edit_sim_en,edit_sim_ru
16852,195236,that 's why you never say a fucking word until...,that 's why you never say a word until you spe...,0.995958,reddit,False,0.9981,True,0.9679,вот почему ты никогда не произносишь ни единог...,"вот почему ты никогда не произносишь ни слова,...",8,18,0.890411,0.812500
1439,79073,"of course , the only majority loser dem is cli...","Of course, the only majority loser is clinton",0.911208,jigsaw,False,0.9387,True,0.9611,"Конечно, единственная демократка, проигравшая ...","Конечно, единственным проигравшим большинством...",8,31,0.846154,0.575342
14601,127468,how in the fuck did bird flu get to antarctica .,How did Bird Flu get to Antarctica?,0.989469,reddit,False,0.9955,True,0.9901,"как, черт возьми, птичий грипп попал в Антаркт...",Как птичий грипп попал в Антарктиду?,18,15,0.625000,0.700000


In [182]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(detox_en2ru_filtered, random_state=1, test_size=500)

In [187]:
def get_detox_task_ru(batch_size=1):
    task = 'detox: '
    sample = detox_en2ru_filtered.sample(batch_size)
    src_id, tgt_id = 'ru_RU', 'ru_RU'
    x = []
    y = []
    for row in sample.itertuples():
        x.append(task + row.toxic_ru)
        y.append(row.neutral_ru)
    return x, y, src_id, tgt_id

get_detox_task_ru(2)

(['detox: зачем нам вообще хотеть видеть его свадьбу? мне на это наплевать',
  'detox: некоторые идеи настолько нагло глупы , что только интеллектуал будет следовать им и верить в них.'],
 ['зачем нам вообще хотеть видеть его свадьбу? Мне все равно',
  'Некоторые идеи настолько нагло невообразимы, что только интеллектуал будет следовать им и верить в них.'],
 'ru_RU',
 'ru_RU')

In [188]:
def get_batch(batch_size=4):
    x = []
    y = []
    bs = batch_size // 4
    for generator in [get_detox_task, get_detox_task_ru, get_paraphrase_task, get_translate_task]:
        for _ in range(bs):
            xx, yy, src_id, tgt_id = generator(1)
            tokenizer.src_lang, tokenizer.tgt_lang = src_id, tgt_id
            x.append(tokenizer(xx))
            with tokenizer.as_target_tokenizer():
                y.append(tokenizer(yy))
    return x, y

In [193]:
batch_size = 8
save_path = '/home/dale/models/detox-parallel/bart-multitask-v2'

In [194]:
model.train()
tq = trange(0, 50_000)
cleanup()

for i in tq:
    x, y = get_batch(batch_size)
    xpad = tokenizer.pad([{k: v[0] for k, v in item.items()} for item in x])
    ypad = tokenizer.pad([{k: v[0] for k, v in item.items()} for item in y])
    
    try:
        labels = torch.tensor(ypad['input_ids'], device=model.device)
        labels[labels==tokenizer.pad_token_id] = -100
        loss = model(
            input_ids=torch.tensor(xpad['input_ids'], device=model.device),
            attention_mask=torch.tensor(xpad['attention_mask'], device=model.device),
            labels=labels,
        ).loss
        loss.backward()
        if i % gradient_steps == 0:
            optimizer.step()
            optimizer.zero_grad(set_to_none=True)
            scheduler.step()
    except RuntimeError as e:
        print('error', i, e)
        loss = None
        optimizer.zero_grad(set_to_none=True)
        cleanup()
        continue

    w = 1 / max(1, min(i, window))
    ewm_loss = ewm_loss * (1-w) + loss.item() * w
    tq.set_description(f'{ewm_loss:3.4f}')

    if i > 0 and i % report_steps == 0:
        print('step', i, 'loss', ewm_loss, )
        if i > 0:
            model.save_pretrained(save_path)
            tokenizer.save_pretrained(save_path)
    if i % cleanup_steps == 0:
        cleanup()

  0%|          | 0/50000 [00:00<?, ?it/s]

error 74 CUDA out of memory. Tried to allocate 658.00 MiB (GPU 0; 10.76 GiB total capacity; 7.49 GiB already allocated; 605.69 MiB free; 9.04 GiB reserved in total by PyTorch)
error 544 CUDA out of memory. Tried to allocate 1.21 GiB (GPU 0; 10.76 GiB total capacity; 7.60 GiB already allocated; 1.01 GiB free; 8.63 GiB reserved in total by PyTorch)
error 748 CUDA out of memory. Tried to allocate 810.00 MiB (GPU 0; 10.76 GiB total capacity; 7.57 GiB already allocated; 747.69 MiB free; 8.90 GiB reserved in total by PyTorch)
step 1000 loss 0.8314551791351633
error 1260 CUDA out of memory. Tried to allocate 894.00 MiB (GPU 0; 10.76 GiB total capacity; 8.03 GiB already allocated; 781.69 MiB free; 8.87 GiB reserved in total by PyTorch)
step 2000 loss 0.8092259806353226
error 2935 CUDA out of memory. Tried to allocate 1.21 GiB (GPU 0; 10.76 GiB total capacity; 8.94 GiB already allocated; 413.69 MiB free; 9.23 GiB reserved in total by PyTorch)
error 2948 CUDA out of memory. Tried to allocate 1.2

In [195]:
model.eval();

In [196]:
print(paraphrase('paraphrase: I like to play with my nice dog.', model, tokenizer))

I like to play with my dog, too.


In [197]:
print(paraphrase('translate: I like to play with my nice dog.', model, tokenizer, tgt_lang='ru_RU'))

Я люблю играть со своей милой собакой.


In [198]:
print(paraphrase('translate: I hate to play with my fucking dog.', model, tokenizer, tgt_lang='ru_RU'))

Я ненавижу играть со своей блядской собакой.


In [199]:
print(paraphrase('translate: Ненавижу играть со своей долбаной собакой.', model, tokenizer, src_lang='ru_RU'))

I hate playing with my fucking dog. Oh, God.


In [200]:
print(paraphrase('paraphrase: I hate to play with my fucking dog.', model, tokenizer, tgt_lang='en_XX'))

I don't like to play with my dog.


In [201]:
print(paraphrase('detox: I hate to play with my fucking dog.', model, tokenizer, tgt_lang='en_XX'))

I don't like to play with my dog.


In [202]:
print(paraphrase('detox: Ненавижу играть со своей долбаной собакой.', model, tokenizer, src_lang='ru_RU', tgt_lang='ru_RU'))

Я ненавижу играть со своей собакой.


In [203]:
test_outputs = [paraphrase('detox: ' + text, model, tokenizer, src_lang='ru_RU', tgt_lang='ru_RU') for text in tqdm(test_inputs)]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [204]:
path = '../results/multitask-1M-translate-yandex/'  # partial training results; need to train further.
if not os.path.exists(path):
    os.mkdir(path)

In [205]:
with open(path + 'results_ru.txt', 'w') as f:
    for line in test_outputs:
        f.write(line + '\n')

Full results (1m steps)  + 50K with Russian translated data
```
python evaluate_ru.py \
    --result_filename scores \
    --input_dir results/multitask-1M-translate-yandex \
    --output_dir results
    
Style accuracy:       0.6192111968994141
Meaning preservation: 0.8775237798690796
Joint fluency:        -0.09076358377933502
Joint score:          -0.04992109537124634
Scores after calibration:
Style accuracy:       0.6572900414466858
Meaning preservation: 0.8162857890129089
Joint fluency:        0.8956218957901001
Joint score:          0.46870917081832886
```